In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import HDBSCAN
from sklearn.cluster import SpectralClustering
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt

In [2]:
with open('image_embeddings.npy', 'rb') as f:
    embeddings = np.load(f)

In [3]:
data = pd.read_csv('data_split_into_classes.csv')

In [4]:
data

,item_id,image_id,title
0,928428250447,9930449149,Беговая дорожка laufstein corsa automatik
1,172952000262,1770341493,Эллиптический тренажер Torneo Vento C-208
2,1064720000255,11289835148,Эллиптический тренажер
3,660092751953,5126102753,Гантели по 10 кг каждая
4,258603750759,2318932240,Kettler Delta XL
...,...,...,...
2451,403658750648,3335234348,Housefir
2452,32427250428,898227519,Силовой комплекс HouseFit 42110
2453,267083000211,2301454396,Степпер decathlon domyos ST320 purple
2454,636204750019,4796160500,В наличи дорожка электрическая 10км/ч нагрузка...


In [1231]:
pca = PCA(n_components=6)
embeddings = pca.fit_transform(embeddings)

In [1232]:
embeddings

array([[ 5.131165  ,  6.3193393 ,  0.01704141, -1.0746912 , -0.3255531 ,
        -0.19851102],
       [ 4.2242403 , -4.7491503 , -0.5864204 , -1.184348  , -3.5193272 ,
         2.1622188 ],
       [ 5.1241555 , -2.896346  ,  0.18441501,  1.3569256 , -4.6768355 ,
         3.027998  ],
       ...,
       [ 1.4754665 ,  0.16168992, -2.7256262 , -1.847764  , -0.0482343 ,
        -0.5083691 ],
       [ 2.062266  ,  3.6966186 , -1.71365   , -4.635372  , -1.7781073 ,
         0.03759253],
       [-4.9890738 ,  0.38849384, -1.4321891 ,  2.6803381 , -2.234838  ,
        -2.1031916 ]], dtype=float32)

In [1322]:
from sklearn.mixture import GaussianMixture

In [1330]:
cl = HDBSCAN(min_cluster_size=82, min_samples=2, cluster_selection_epsilon=0.5)
cl1 = GaussianMixture(n_components=7)

In [1331]:
predict_img = cl.fit_predict(embeddings)
predict_img1 = cl1.fit_predict(embeddings)

In [1369]:
mapping = {
    0: 0,
    1: 4,
    4: 6,
    6: -1,
    2: 1,
    3: 3,
    5: 2
}

mapped_func = np.vectorize(lambda x: mapping[x])

In [1370]:
predict_img1 = mapped_func(predict_img1)

In [1372]:
data[predict_img == 2]['title']

14                                    Велотренажер Torneo
18                    Продам велотренажер Sven Sport, б/у
70                               Велотренажер Carbon U407
71      Электромагнитный велотренажер Oxygen Fitness s...
75                      Продам Велотренажер torneo 6000 р
                              ...                        
2379                            Велотренажёр Domyos FC650
2404                                 Велотренажёр Rroteus
2417                       Велотренажор torneo B-352 Vita
2435                   Продам велотренажер. Цена 7000 тыс
2451                                             Housefir
Name: title, Length: 253, dtype: object

In [938]:
data['title']

0               Беговая дорожка laufstein corsa automatik
1               Эллиптический тренажер Torneo Vento C-208
2                                  Эллиптический тренажер
3                                 Гантели по 10 кг каждая
4                                        Kettler Delta XL
                              ...                        
2451                                             Housefir
2452                      Силовой комплекс HouseFit 42110
2453                Степпер decathlon domyos ST320 purple
2454    В наличи дорожка электрическая 10км/ч нагрузка...
2455    Комплект штанга прямая + 2 гантели вес 55кг(арт1)
Name: title, Length: 2456, dtype: object

In [944]:
import re
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText

WORD_PATTERN = '(?u)\\b\\w\\w+\\b'

reg_exp = re.compile(pattern=WORD_PATTERN)
w2v_train_data = pd.read_csv('more_titles.csv',  index_col=0)
print(w2v_train_data.shape)
w2v_train_data.head()

(97860, 1)


,title
item_id,
1114484500204,Спортивный снаряд
263752273290,Эллиптический тренажер С-507
1107614001153,"Батут Hasttings Air Game 3,05 м 10FT"
726016250652,Гриф и блины Barbell
62094000446,Обруч массажный разборный BD45


In [945]:
sentences = [reg_exp.findall(s.lower()) for s in w2v_train_data.title]
sentences[:5]

[['спортивный', 'снаряд'],
 ['эллиптический', 'тренажер', '507'],
 ['батут', 'hasttings', 'air', 'game', '05', '10ft'],
 ['гриф', 'блины', 'barbell'],
 ['обруч', 'массажный', 'разборный', 'bd45']]

In [946]:
from gensim.models.callbacks import CallbackAny2Vec

In [947]:
class LossLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
        self.loss_previous_step = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        print('Loss after epoch {}: {}'.format(self.epoch, loss - self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss


class EpochLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        print(f'Epoch {self.epoch}')
        self.epoch += 1


In [1028]:
fasttext = FastText(sg=1, vector_size=32, window=4, min_count=10, hs=1, negative=12, min_n=5, max_n=10)
fasttext.build_vocab(sentences)
fasttext.train(
    sentences,
    total_examples=fasttext.corpus_count,
    epochs=50,
    compute_loss=True,
    callbacks=[LossLogger()]
)

Loss after epoch 0: 0.0
Loss after epoch 1: 0.0
Loss after epoch 2: 0.0
Loss after epoch 3: 0.0
Loss after epoch 4: 0.0
Loss after epoch 5: 0.0
Loss after epoch 6: 0.0
Loss after epoch 7: 0.0
Loss after epoch 8: 0.0
Loss after epoch 9: 0.0
Loss after epoch 10: 0.0
Loss after epoch 11: 0.0
Loss after epoch 12: 0.0
Loss after epoch 13: 0.0
Loss after epoch 14: 0.0
Loss after epoch 15: 0.0
Loss after epoch 16: 0.0
Loss after epoch 17: 0.0
Loss after epoch 18: 0.0
Loss after epoch 19: 0.0
Loss after epoch 20: 0.0
Loss after epoch 21: 0.0
Loss after epoch 22: 0.0
Loss after epoch 23: 0.0
Loss after epoch 24: 0.0
Loss after epoch 25: 0.0
Loss after epoch 26: 0.0
Loss after epoch 27: 0.0
Loss after epoch 28: 0.0
Loss after epoch 29: 0.0
Loss after epoch 30: 0.0
Loss after epoch 31: 0.0
Loss after epoch 32: 0.0
Loss after epoch 33: 0.0
Loss after epoch 34: 0.0
Loss after epoch 35: 0.0
Loss after epoch 36: 0.0
Loss after epoch 37: 0.0
Loss after epoch 38: 0.0
Loss after epoch 39: 0.0
Loss after

(14894359, 22169750)

In [1138]:
w2v_model = Word2Vec(sg=0, vector_size=6, window=3, min_count=10, hs=1, negative=12)
w2v_model.build_vocab(sentences)
w2v_model.train(
    sentences,
    total_examples=w2v_model.corpus_count,
    epochs=50,
    compute_loss=True,
    callbacks=[LossLogger()]
)

Loss after epoch 0: 1076067.5
Loss after epoch 1: 927750.0
Loss after epoch 2: 893838.0
Loss after epoch 3: 845985.75
Loss after epoch 4: 848857.25
Loss after epoch 5: 817321.0
Loss after epoch 6: 816613.5
Loss after epoch 7: 815408.5
Loss after epoch 8: 811777.5
Loss after epoch 9: 829621.0
Loss after epoch 10: 864072.0
Loss after epoch 11: 867310.0
Loss after epoch 12: 867968.0
Loss after epoch 13: 924684.0
Loss after epoch 14: 867300.0
Loss after epoch 15: 866629.0
Loss after epoch 16: 866555.0
Loss after epoch 17: 868106.0
Loss after epoch 18: 867150.0
Loss after epoch 19: 515972.0
Loss after epoch 20: 384228.0
Loss after epoch 21: 381202.0
Loss after epoch 22: 363782.0
Loss after epoch 23: 378602.0
Loss after epoch 24: 376522.0
Loss after epoch 25: 376704.0
Loss after epoch 26: 373316.0
Loss after epoch 27: 374218.0
Loss after epoch 28: 370926.0
Loss after epoch 29: 368270.0
Loss after epoch 30: 368884.0
Loss after epoch 31: 366814.0
Loss after epoch 32: 365400.0
Loss after epoch 

(14894818, 22169750)

In [1126]:
class Word2VecTransformer:

    def __init__(self, w2v_model, word_pattern):

        self.w2v_model = w2v_model
        self.word_pattern = word_pattern
        self.re = re.compile(pattern=self.word_pattern)

    def fit(self, X):
        return self

    def transform(self, X):

        X_transformed = np.zeros((len(X), self.w2v_model.wv.vector_size))
        for i, title in enumerate(X):

            title_vector = np.zeros((self.w2v_model.wv.vector_size,))
            tokens = self.re.findall(title.lower())
            for token in tokens:
                if token in self.w2v_model.wv.key_to_index:
                    title_vector = np.max([self.w2v_model.wv.get_vector(token), title_vector], axis=0)

            X_transformed[i] = title_vector

        return X_transformed

In [1139]:
ft_transformer = Word2VecTransformer(w2v_model=w2v_model, word_pattern=WORD_PATTERN)

text_embedding = ft_transformer.transform(data['title'].values)

In [1117]:
data['title']

0               Беговая дорожка laufstein corsa automatik
1               Эллиптический тренажер Torneo Vento C-208
2                                  Эллиптический тренажер
3                                 Гантели по 10 кг каждая
4                                        Kettler Delta XL
                              ...                        
2451                                             Housefir
2452                      Силовой комплекс HouseFit 42110
2453                Степпер decathlon domyos ST320 purple
2454    В наличи дорожка электрическая 10км/ч нагрузка...
2455    Комплект штанга прямая + 2 гантели вес 55кг(арт1)
Name: title, Length: 2456, dtype: object

In [1099]:
text_embedding = pca.fit_transform(text_embedding)

In [1140]:
w2v_model.wv.similar_by_word('тренажер')

[('тренажёр', 0.9797071814537048),
 ('315', 0.9789050817489624),
 ('типа', 0.9765779376029968),
 ('треножер', 0.9679059386253357),
 ('hd', 0.9651532769203186),
 ('325', 0.9591271281242371),
 ('hammer', 0.9483343362808228),
 ('вм', 0.9472570419311523),
 ('гребля', 0.9471397995948792),
 ('вакуумный', 0.946445107460022)]

In [1141]:
from sklearn.cluster import DBSCAN, OPTICS

cl_text = DBSCAN(min_samples=5, n_jobs=4)

predict_text = cl_text.fit_predict(text_embedding)

In [1145]:
data[predict_text == 2]['title']

7                                                Гиря 16
17                                   Гири б/у 32, 24, 16
85                                               Гиря 32
109                                            Гиря 32кг
215                                  Гири 16-32кг парные
236                                            Гиря 24кг
396                                      Гиря 24кг, СССР
424                                            Гира 24кг
520                                         Гири по 24кг
527                                       Гиря неваляшка
581                               Гири, гантели. 24-16кг
586                             Комплект Гирь (16,24,32)
618                                              Гири 16
686                                         Гири 24 и 32
737                                      Гири на 16 и 24
1012                                     Гири 16, 20, 32
1040                                      Гиря 16кг cccр
1145                           

In [272]:
np.unique(predict_img)

array([-1,  0,  1,  2,  3,  4], dtype=int64)

In [323]:
data[predict_img == 2]['title'].sample(10)

1390                                      Продам элип
1538                   Велотренажер Hasttings Wega S3
610                                   Продаю тренажер
28                         Эллиптический тренажёр б/у
1971    Вертикальный велоэргометр Vision U60 (U60-03)
1416                  Велотренажер sport elit se-1311
2261             Эллиптический тренажёр Torneo C-208G
804               Велотренажер Body Sculpture BC-5710
1052                      Эллиптический тренажер Q300
2268             Эллиптический тренажер pumori BY-430
Name: title, dtype: object

In [352]:
data[predict_text == 2]['title']

3                                 Гантели по 10 кг каждая
7                                                 Гиря 16
11                     Гантели разборные новые 2 по 30 кг
12                                   Советская гиря 24 кг
15                                       Чугунные гантели
                              ...                        
2446    Гантели неразборные виниловые, розовые 2шт. по...
2447            Лавка штанга гриф для гантели блины 60 кг
2448                                Гантели Barbell 24 кг
2449                                        Гиря на 22 кг
2455    Комплект штанга прямая + 2 гантели вес 55кг(арт1)
Name: title, Length: 607, dtype: object

In [518]:
data[predict_img == 4]['title']

3                                 Гантели по 10 кг каждая
15                                       Чугунные гантели
22                                           Штанга 50 кг
23                                            Утяжелители
32                         Гантели 33кг новые с доставкой
                              ...                        
2433                       Гантели СССР по 12. Клеймо пмз
2441           Разборные гантели MB barbell atlet Д26. 37
2446    Гантели неразборные виниловые, розовые 2шт. по...
2448                                Гантели Barbell 24 кг
2455    Комплект штанга прямая + 2 гантели вес 55кг(арт1)
Name: title, Length: 475, dtype: object

In [366]:
mapping = {
    -1: -1,
    0: 0,
    1: 2,
    2: 1
}

mapped_func = np.vectorize(lambda x: mapping[x])

In [367]:
mapped_func(predict_text)

array([-1, -1,  2, ..., -1,  0,  1])

In [369]:
predict_img[predict_img == -1] = mapped_func(predict_text[predict_img == -1])

In [831]:
data[predict_img == -1]['title']

2294         Беговая дорожка BH fitness pioneer R2
2297        Беговая дорожка Clear Fit Eco ET 20 AI
2406    Продается Беговая Дорожка Torneo Linia 201
Name: title, dtype: object

In [836]:
predict_img

array([0, 2, 2, ..., 3, 0, 4], dtype=int64)

In [837]:
mapping = {
    0: 0,
    2: 1,
    4: 2,
    3: 3,
    1: 4
}

mapped_func = np.vectorize(lambda x: mapping[x])

In [838]:
mapped_func(predict_img)

array([0, 1, 1, ..., 3, 0, 2])

In [839]:
pd.DataFrame(mapped_func(predict_img)).to_csv('solution.csv', header=['cluster'], index=False)

In [1156]:
data[predict_img == 4]['title']

3                                 Гантели по 10 кг каждая
11                     Гантели разборные новые 2 по 30 кг
13                                              Mi band 2
15                                       Чугунные гантели
21                                    Обруч для похудения
                              ...                        
2441           Разборные гантели MB barbell atlet Д26. 37
2446    Гантели неразборные виниловые, розовые 2шт. по...
2448                                Гантели Barbell 24 кг
2450              Kettler Basic Wrist утяжелители для рук
2455    Комплект штанга прямая + 2 гантели вес 55кг(арт1)
Name: title, Length: 604, dtype: object

In [1157]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6)

In [1158]:
predict_kmeans = kmeans.fit_transform(embeddings)

C:\Users\aleks\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [1161]:
predict_kmeans = predict_kmeans.argmin(axis=1)

In [1208]:
mapping = {
    0: -1,
    1: 4,
    2: 0,
    3: 5,
    4: 3,
    5: 1
}

mapped_func = np.vectorize(lambda x: mapping[x])

In [1209]:
predict_kmeans = mapped_func(predict_kmeans)

In [1211]:
predict_kmeans

array([3, 0, 0, ..., 3, 3, 4])

In [1210]:
data[predict_kmeans == 0]['title'].sample(10)

2023            Продам домашний велотренажер Domyos
873                         Тренажёр Cardio Twister
1810    Эллиптический тренажер kettler axos cross m
2113                 Велотренажер oxygen concept iv
1406                      Велотренажер Atemi AC 701
1060          Эллиптический тренажер Hasttings Q600
945                        Велотренажер мини Torneo
553        Велотренажер Torneo в отличном состоянии
1308                         Магнитный велотренажер
933                         Велотренажер Sport Elit
Name: title, dtype: object

In [1212]:
data[predict_img == 0]['title']

1       Эллиптический тренажер Torneo Vento C-208
2                          Эллиптический тренажер
5                   Эллиптический тренажер торнео
16                Эллиптический тренажер Sole E95
25                                       Тренажёр
                          ...                    
2376                 Тренажер эллиптический atemi
2388                 Эллиптический тренажер Rally
2395                         Элепический тренажёр
2415                              Тренажер Хаммер
2416              Эллиптический тренажер Appalget
Name: title, Length: 233, dtype: object

In [1216]:
predict_img[predict_img == -1] = predict_kmeans[predict_img == -1]

In [1219]:
data[predict_img == 0]['title']

1       Эллиптический тренажер Torneo Vento C-208
2                          Эллиптический тренажер
5                   Эллиптический тренажер торнео
16                Эллиптический тренажер Sole E95
25                                       Тренажёр
                          ...                    
2395                         Элепический тренажёр
2415                              Тренажер Хаммер
2416              Эллиптический тренажер Appalget
2421            Тренажер для пресса torneo rabbit
2424       Велотренажер магнитный kpower кlj 8304
Name: title, Length: 329, dtype: object

In [1220]:
new_predict_img = predict_img

In [1221]:
new_predict_img[new_predict_img == -1] = 0

In [1222]:
data[new_predict_img == 0]['title']

1       Эллиптический тренажер Torneo Vento C-208
2                          Эллиптический тренажер
5                   Эллиптический тренажер торнео
9        Практический новый элиптический тренажер
16                Эллиптический тренажер Sole E95
                          ...                    
2415                              Тренажер Хаммер
2416              Эллиптический тренажер Appalget
2421            Тренажер для пресса torneo rabbit
2424       Велотренажер магнитный kpower кlj 8304
2445                    Велотренажер Domyos VM200
Name: title, Length: 585, dtype: object

In [1224]:
mapping = {
    3: 0,
    0: 1,
    4: 2,
    1: 3,
    5: 4,
    2: 5
}

mapped_func = np.vectorize(lambda x: mapping[x])

In [1225]:
new_predict_img = mapped_func(new_predict_img)

In [1226]:
pd.DataFrame(new_predict_img).to_csv('solution.csv', header=['cluster'], index=False)

In [1373]:
predict_img[predict_img == -1] = predict_img1[predict_img == -1]

In [1386]:
predict_img1

array([0, 0, 0, ..., 3, 3, 6])

In [1384]:
data[predict_img == 3]

,item_id,image_id,title
24,471878500455,3710743695,Oxygen villa deluxe II ML HRC беговая дорожка
27,416560500063,3442644360,Тренажер Torneo
51,1124019250757,11852507484,Беговая дорожка электрическая Carbon T 507
52,832277750106,8616217817,Беговая дорожка Xiaomi walking pad
63,820968750757,8480886715,Продам беговую дорожку Torneo Magic T-410
...,...,...,...
2432,718981500233,5995119516,Беговая дорожка электрическая Stingray ST-9315
2434,590999000268,4516499140,Беговая дорожка AppleGate T40 ADC купить в Омске
2443,98388251715,1366170222,Беговая дорожка Hasttings Evok
2453,267083000211,2301454396,Степпер decathlon domyos ST320 purple


In [1387]:
predict_img[predict_img == -1] = predict_img1[predict_img == -1]

In [1389]:
mapping = {
    0: 0,
    6: 1,
    1: 2,
    4: 3,
    3: 4,
    5: 5,
    2: 6
}

mapped_func = np.vectorize(lambda x: mapping[x])

predict_img = mapped_func(predict_img)

In [1390]:
predict_img

array([0, 0, 0, ..., 4, 4, 1])

In [1391]:
pd.DataFrame(predict_img).to_csv('solution.csv', header=['cluster'], index=False)

In [5]:
import faiss

In [6]:
embeddings

array([[ 5.1311584e+00,  6.3193374e+00,  1.7005675e-02, ...,
         1.2844603e-01,  2.9735411e-02, -5.3486533e-02],
       [ 4.2242427e+00, -4.7491603e+00, -5.8638811e-01, ...,
         5.1246941e-02,  2.4692614e-02, -6.6261292e-02],
       [ 5.1241570e+00, -2.8963387e+00,  1.8442334e-01, ...,
         2.2176186e-02,  5.8165304e-02,  3.9022826e-02],
       ...,
       [ 1.4754670e+00,  1.6166554e-01, -2.7256250e+00, ...,
        -5.1724133e-03, -6.8166345e-02,  1.6987930e-01],
       [ 2.0622656e+00,  3.6965909e+00, -1.7136641e+00, ...,
         1.1754863e-01, -2.3782093e-02,  3.9456457e-02],
       [-4.9890742e+00,  3.8849327e-01, -1.4322029e+00, ...,
        -2.2285390e-03, -1.8312924e-01, -1.0697322e-01]], dtype=float32)

In [22]:
ncentroids = 7
niter = 20
verbose = True
d = 512
kmeans = faiss.Kmeans(d, ncentroids, niter=niter, verbose=verbose)
kmeans.train(embeddings)

135632.984375

In [23]:
D, I = kmeans.index.search(embeddings, 1)

In [36]:
I

array([[6],
       [1],
       [1],
       ...,
       [5],
       [6],
       [2]], dtype=int64)

In [69]:
res = pd.DataFrame(I)

In [70]:
mapping = {
    6: 0,
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    0: 6
}

In [72]:
res[0]

0       6
1       1
2       1
3       2
4       3
       ..
2451    0
2452    3
2453    5
2454    6
2455    2
Name: 0, Length: 2456, dtype: int64

In [75]:
res = res[0].map(lambda x: mapping[x])

In [77]:
res.to_csv('solution.csv', header=['cluster'], index=False)